# TP ChromaDB
    
Ce notebook guide vos étapes pour découvrir et pratiquer ChromaDB (base de données vectorielle).
Il est structuré en 4 séquences. Remplissez les cellules TODO.
    
**Prérequis** : Python 3.13+
    
> Astuce : Exécutez chaque cellule dans l’ordre. Documentez vos observations dans les zones prévues.


---
## 0) Préparation de l'environnement

Objectifs :
- Installer les dépendances nécessaires : `chromadb` et un encodeur d'embeddings local (ex : `sentence-transformers`).
- Vérifier les versions.


In [1]:
# TODO
import chromadb

---
## 1) Découverte et premier index

**Objectifs**
- Comprendre le concept d'embeddings et de recherche de similarité.
- Créer un client ChromaDB, une collection, ajouter quelques documents, interroger.


In [4]:
# Créez un client avec une base de données persistances
# TODO
chroma_client = chromadb.PersistentClient()

In [5]:
# Créez une collection (<=> table en sgbdr traditionnel)
# TODO
collection = chroma_client.create_collection(name="my_collection")

In [6]:
# Déclarez 3 variables textuelles à ajouter à notre collection
# TODO
doc1 = "La préfecture de La Réunion se situe à Saint-Denis."
doc2 = "ChromaDB est une base de données vectorielle simple à utiliser pour la recherche sémantique."
doc3 = "Le piton de la Fournaise est l'un des volcans les plus actifs du monde."

In [15]:
# Déclarez les metadonnées et les identifiants uniques
# TODO
docs = [doc1, doc2, doc3]
ids = ["id1", "id2", "id3"]
metadatas = [
    {"description": "Article sur l'île de la Réunion", "version": "1.0", "created_by": "admin"},
    {"description": "Article sur les bases de données vectorielle", "version": "1.1", "created_by": "admin"},
    {"description": "Article sur l'île de la Réunion", "version": "1.2", "created_by": "admin"}
]

In [16]:
# Ajoutez les données/metadonnées/identifiants à la collection
# TODO
collection.add(
    ids=ids,
    documents=docs,
    metadatas=metadatas
)

/home/anna/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:26<00:00, 3.20MiB/s]


In [21]:
# Effectuez une recherche de similitude sur une question en rapport avec un des thèmes de vos documents
# TODO
results = collection.query(
    query_texts=["un query sur les information sur l'île de la Réunion"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
results

{'ids': [['id_4', 'id1']],
 'embeddings': None,
 'documents': [["Le tourisme est un secteur économique majeur sur l'île de La Réunion.",
   'La préfecture de La Réunion se situe à Saint-Denis.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'version': '1.0',
    'created_by': 'admin',
    'description': 'Article sur La Réunion - Doc 4'},
   {'version': '1.0',
    'description': "Article sur l'île de la Réunion",
    'created_by': 'admin'}]],
 'distances': [[0.631873607635498, 0.6965787410736084]]}

> **Observation (à compléter)**  
- Qu'ai-je observé sur les résultats (ordre, distances, ids) ?  
- À quoi servent les embeddings ?
- Quel est l'embedder par défaut utilisé par chromadb ?


Les résultat du query sont pertinents, il envoie toutes les informations liées au résultats trouvés. Les distances sont float et semble longues aprés la virgule, est-ce signifiant?
Les embeddings sevent à faire les meilleures réponses au query avec les données dans la collection choisit.

---
## 2) Métadonnées, filtrage, mise à jour et suppression

**Objectifs**
- Ajouter un jeu de documents plus riche (10–15+).
- Utiliser des métadonnées et effectuer des recherches avec filtres.
- Mettre à jour et supprimer des entrées.


In [33]:
# Créez ou chargez un corpus de 10–15 documents
# Exemple : fabriquez un corpus simple ici. Vous pouvez remplacer par vos propres textes.
# TODO
more_docs = [
    "La Réunion est une île française située dans l'océan Indien.",
    "Le piton de la Fournaise est un volcan actif de La Réunion.",
    "La cuisine réunionnaise mélange des influences africaines, indiennes et européennes.",
    "Le tourisme est un secteur économique majeur sur l'île de La Réunion.",
    "Les bases de données vectorielles sont utilisées pour stocker et interroger des vecteurs d'embeddings.",
    "Les embeddings permettent de représenter des documents ou des mots sous forme de vecteurs, facilitant ainsi la recherche sémantique.",
    "Une base de données vectorielle est optimisée pour des recherches rapides et efficaces dans des espaces de haute dimension.",
    "FAISS (Facebook AI Similarity Search) est une bibliothèque qui permet de rechercher des vecteurs dans de grands ensembles de données.",
    "Chroma est une base de données open-source spécialisée dans les embeddings pour la recherche et l'indexation de données vectorielles.",
    "Les applications des bases de données vectorielles incluent la recherche d'images similaires, la recherche sémantique et la recommandation de contenu."
]

# Créez des identifiants uniques pour chaque document
more_ids = [f"id_{i+1}" for i in range(len(more_docs))]


# Créez des métadonnées associées à chaque document
creators = ["admin", "user1", "user2"]
more_metadatas = [
    {
        "version": "1.0",
        "description": f"Article sur La Réunion - Doc {i+1}" if i < 4 else f"Article sur bases de données vectorielles - Doc {i+1}",
        "created_by": creators[i % len(creators)]  # Attribue un créateur différent à chaque document    }
    }
    for i in range(len(more_docs))
]

In [34]:
# Ajouter de nouveaux documents
# TODO
collection.add(
    ids=more_ids,
    documents=more_docs,
    metadatas=more_metadatas
)

In [35]:
collection.peek()

{'ids': ['id1',
  'id2',
  'id3',
  'id_1',
  'id_2',
  'id_3',
  'id_4',
  'id_5',
  'id_6',
  'id_7'],
 'embeddings': array([[-0.04831972,  0.01283783, -0.02138843, ...,  0.00970293,
         -0.0125578 , -0.0198486 ],
        [-0.10025828, -0.00313545, -0.04016039, ...,  0.10073496,
          0.05870798, -0.07088163],
        [-0.02009018,  0.01328702, -0.02889018, ...,  0.03941492,
          0.06404133,  0.04178637],
        ...,
        [-0.04002536, -0.01982269, -0.0090268 , ..., -0.044636  ,
          0.06083874, -0.01036558],
        [-0.0497831 ,  0.02307898, -0.00789293, ...,  0.05138446,
          0.03775414, -0.00517705],
        [-0.01471932, -0.02846535, -0.01509933, ...,  0.01643618,
          0.0247216 , -0.03628836]], shape=(10, 384)),
 'documents': ['La préfecture de La Réunion se situe à Saint-Denis.',
  'ChromaDB est une base de données vectorielle simple à utiliser pour la recherche sémantique.',
  "Le piton de la Fournaise est l'un des volcans les plus actifs du m

In [36]:
# Effectuez une recherche sans filtre
# TODO
results = collection.query(
    query_texts=["un query sur les information sur les bases de données vectorielle"], # Chroma will embed this for you
    n_results=3 # how many results to return
)
results

{'ids': [['id_5', 'id_7', 'id_10']],
 'embeddings': None,
 'documents': [["Les bases de données vectorielles sont utilisées pour stocker et interroger des vecteurs d'embeddings.",
   'Une base de données vectorielle est optimisée pour des recherches rapides et efficaces dans des espaces de haute dimension.',
   "Les applications des bases de données vectorielles incluent la recherche d'images similaires, la recherche sémantique et la recommandation de contenu."]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'version': '1.0',
    'created_by': 'admin',
    'description': 'Article sur bases de données vectorielles - Doc 5'},
   {'version': '1.0',
    'description': 'Article sur bases de données vectorielles - Doc 7',
    'created_by': 'admin'},
   {'description': 'Article sur bases de données vectorielles - Doc 10',
    'created_by': 'admin',
    'version': '1.0'}]],
 'distances': [[0.40470319986343384, 0.6897591352462769, 0.789898

In [37]:
# Effectuez une recherche avec filtre métadonnée (ex.: seulement 'technique')
# TODO
results = collection.query(
    query_texts=["un query sur les information sur les bases de données vectorielle"], # Chroma will embed this for you
    n_results=2, # how many results to return
    where={"created_by":"admin"}  # Filtrage via les métadonnées
)
results

{'ids': [['id_5', 'id_7']],
 'embeddings': None,
 'documents': [["Les bases de données vectorielles sont utilisées pour stocker et interroger des vecteurs d'embeddings.",
   'Une base de données vectorielle est optimisée pour des recherches rapides et efficaces dans des espaces de haute dimension.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'created_by': 'admin',
    'description': 'Article sur bases de données vectorielles - Doc 5',
    'version': '1.0'},
   {'created_by': 'admin',
    'description': 'Article sur bases de données vectorielles - Doc 7',
    'version': '1.0'}]],
 'distances': [[0.40470319986343384, 0.6897591352462769]]}

In [38]:
# Effectuez une mise à jour d'un document (update)
# TODO
new_metadata = {
    "version": "1.1",  # Mettre à jour la version
    "description": "Mise à jour de l'article sur La Réunion - Document modifié",  # Nouvelle description
    "created_by": "user1"  # On peut aussi changer le créateur si nécessaire
}
collection.upsert(
    documents=["Anna est originaire de l'île de La Réunion"],  # Nouveau contenu du document
    metadatas=[new_metadata],  # Nouvelle métadonnée
    ids=["id_3"]  # ID du document à mettre à jour
)

result = collection.query(
    query_texts=["d'où est Anna originaire"],  # Recherche basée sur le nouvel embedding
    n_results=1
)

result

{'ids': [['id_3']],
 'embeddings': None,
 'documents': [["Anna est originaire de l'île de La Réunion"]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'description': "Mise à jour de l'article sur La Réunion - Document modifié",
    'version': '1.1',
    'created_by': 'user1'}]],
 'distances': [[0.5085338354110718]]}

In [41]:
# Procédez à une suppression d'un document
# TODO
collection.delete(ids=["id_3"])

results = collection.query(
    query_texts=["d'où est Anna originaire"],  # Embedding factice pour la requête
    n_results=1
)

results

{'ids': [['id_1']],
 'embeddings': None,
 'documents': [["La Réunion est une île française située dans l'océan Indien."]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'description': 'Article sur La Réunion - Doc 1',
    'version': '1.0',
    'created_by': 'admin'}]],
 'distances': [[1.0031177997589111]]}

> **Synthèse (à compléter)**  
- Dans quels cas les filtres par métadonnées m'ont aidé ?  
- Quel impact des mises à jour/suppressions sur les résultats ?


Pour filtrer les créateur, update les résultats.

---
## 3) Cas pratique : recherche sémantique sur mini‑corpus

**Objectifs**
- Construire un corpus sur un thème choisi.
- Lancer plusieurs requêtes et analyser la pertinence.
- (Bonus) Chunking si les documents sont longs.


In [43]:
# Remplacez par votre thème/corpus (ex.: FAQ interne, fiches produits, extraits d'articles)
# TODO:
client = chromadb.Client()

theme_docs = [
    "Kurt Cobain, le leader du groupe Nirvana, est l'une des icônes les plus marquantes du rock des années 90. Né en 1967 à Aberdeen, Washington, il est devenu célèbre avec l'album 'Nevermind'.",
    "Kurt Cobain s'est suicidé en 1994, laissant derrière lui un héritage musical qui influence encore de nombreux artistes aujourd'hui. Son style unique a redéfini le grunge.",
    "La musique de Kurt Cobain se caractérise par une grande émotion brute, souvent marquée par la mélancolie et la douleur. Les chansons de Nirvana, comme 'Smells Like Teen Spirit', sont devenues des hymnes de la génération X.",
    "Kurt Cobain a toujours été une figure de contradiction, cherchant à éviter la célébrité tout en étant l'une des plus grandes stars du rock de son époque. Son combat contre la dépendance et ses problèmes de santé mentale sont au cœur de sa musique.",
    "Malgré sa courte carrière, Kurt Cobain reste une figure emblématique, souvent citée comme l'un des plus grands chanteurs et compositeurs du rock moderne. Son influence se fait encore sentir dans de nombreux genres musicaux."
]

theme_ids = [f"cobain_doc_{i+1}" for i in range(len(theme_docs))]

theme_metas = [
    {"version": "1.0", "description": "Biographie de Kurt Cobain", "created_by": "anna"},
    {"version": "1.0", "description": "Analyse de l'impact de Kurt Cobain sur le grunge", "created_by": "anna"},
    {"version": "1.0", "description": "Analyse de la musique de Kurt Cobain et de Nirvana", "created_by": "anna"},
    {"version": "1.0", "description": "Exploration de la vie et des luttes de Kurt Cobain", "created_by": "anna"},
    {"version": "1.0", "description": "L'héritage de Kurt Cobain dans la musique moderne", "created_by": "anna"}
]

collection_KC = client.get_or_create_collection(name="kurt_cobain")

collection_KC.add(
    documents=theme_docs, 
    ids=theme_ids, 
    metadatas=theme_metas
    )

print("Collection 'kurt_cobain' prête avec", collection_KC.count(), "documents.")

# Exemple d'ajout (décommentez quand prêts)
# collection_theme = client.get_or_create_collection(name="theme")
# collection_theme.add(documents=theme_docs, ids=theme_ids, metadatas=theme_metas)
# print("Collection 'theme' prête avec", collection_theme.count(), "documents.")

Collection 'kurt_cobain' prête avec 5 documents.


In [44]:
# TODO: Définissez 3–5 requêtes utilisateur et interrogez la collection
# queries = ["...", "...", "..."]
# for q in queries:
#     res = collection_theme.query(query_texts=[q], n_results=3)
#     print("\nQuery:", q)
#     print(json.dumps(res, indent=2, ensure_ascii=False))

import json

queries = ["année de décé de Kurt Cobain", "type de music Kurt Cobain", "nom du groupe de Kurt Cobain"]
for q in queries:
    res = collection_KC.query(query_texts=[q], n_results=3)
    print("\nQuery:", q)
    print(json.dumps(res, indent=2, ensure_ascii=False))


Query: année de décé de Kurt Cobain
{
  "ids": [
    [
      "cobain_doc_2",
      "cobain_doc_4",
      "cobain_doc_1"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "Kurt Cobain s'est suicidé en 1994, laissant derrière lui un héritage musical qui influence encore de nombreux artistes aujourd'hui. Son style unique a redéfini le grunge.",
      "Kurt Cobain a toujours été une figure de contradiction, cherchant à éviter la célébrité tout en étant l'une des plus grandes stars du rock de son époque. Son combat contre la dépendance et ses problèmes de santé mentale sont au cœur de sa musique.",
      "Kurt Cobain, le leader du groupe Nirvana, est l'une des icônes les plus marquantes du rock des années 90. Né en 1967 à Aberdeen, Washington, il est devenu célèbre avec l'album 'Nevermind'."
    ]
  ],
  "uris": null,
  "included": [
    "metadatas",
    "documents",
    "distances"
  ],
  "data": null,
  "metadatas": [
    [
      {
        "version": "1.0",
        "descripti

> **Analyse (à compléter)**  
Pour chaque requête :
- Les documents renvoyés sont‑ils pertinents ? Pourquoi ?  
- Les distances/scores reflètent‑ils votre intuition ?


Les documents renvoyés sont pertinents. Je n'avais pas d'intuition particuliére sur le scores/distances.

---
## 4) Extension : mini‑application & bonnes pratiques

**Objectifs**
- Construire une petite interface d'interrogation en console.
- Persister les données et discuter des bonnes pratiques.

**Livrable**
- Script/Notebook de mini‑app + fiche « bonnes pratiques » (1 page).


In [52]:
# Mini‑app console (simple)
COLLECTION_NAME = "kurt_cobain_2"
# Utilise la collection_KC2 de base. Tapez 'quit' pour sortir.
print("Mini‑app de recherche (collection_KC2:", COLLECTION_NAME, ")")
# TODO

import chromadb
import json

# Création du client pour ChromaDB
client = chromadb.Client()

# Création ou récupération de la collection_KC2
collection_KC2 = client.get_or_create_collection(name="kurt_cobain_2")

# Exemple de documents sur Kurt Cobain
theme_docs = [
    "Kurt Cobain, le leader du groupe Nirvana, est souvent perçu comme une icône de la musique grunge. Né en 1967 à Aberdeen, Washington, il a fondé Nirvana avec le bassiste Krist Novoselic.",
    "L'album 'Nevermind', sorti en 1991, a propulsé Nirvana au rang de superstars internationales. La chanson 'Smells Like Teen Spirit' est rapidement devenue un hymne générationnel.",
    "Kurt Cobain a toujours revendiqué une approche musicale honnête et brute, où il exprimait ses émotions personnelles. La scène musicale grunge a explosé grâce à des groupes comme Nirvana, Pearl Jam, et Soundgarden.",
    "Le titre 'Smells Like Teen Spirit' est l'un des morceaux les plus emblématiques de Nirvana. Il est souvent cité comme l'une des chansons les plus importantes des années 1990.",
    "En plus de sa carrière musicale, Kurt Cobain était aussi un artiste visuel. Il a réalisé plusieurs dessins et peintures pendant sa vie.",
    "Kurt Cobain se battait contre des problèmes de santé mentale, y compris la dépression et l'anxiété, ce qui a influencé de nombreuses chansons de Nirvana.",
    "L'album 'In Utero', sorti en 1993, a marqué une évolution musicale pour Nirvana. Produit par Steve Albini, il est plus sombre et expérimental que 'Nevermind'.",
    "Le suicide de Kurt Cobain en avril 1994 a choqué le monde entier. Il a laissé derrière lui un héritage musical immense, mais aussi une série de questions non résolues sur sa santé mentale et ses luttes personnelles.",
    "Malgré sa disparition, l'impact de Kurt Cobain sur la musique et la culture populaire reste profond. Il est toujours considéré comme l'une des figures les plus importantes du rock des années 1990.",
    "Kurt Cobain était également un militant de la liberté d'expression et un critique acerbe de l'industrie musicale. Il a souvent critiqué l'exploitation commerciale de la musique.",
    "La chanson 'Come as You Are', de l'album 'Nevermind', est une des chansons les plus emblématiques de Nirvana. Elle évoque le concept de l'authenticité et de l'acceptation de soi.",
    "Kurt Cobain était marié à Courtney Love, la chanteuse de Hole. Leur relation a souvent été médiatisée et controversée, ce qui a ajouté une pression supplémentaire à sa vie personnelle.",
    "Le groupe Nirvana a joué un rôle clé dans la démocratisation du genre musical grunge, un mélange de punk et de rock alternatif, qui est devenu un phénomène mondial dans les années 90.",
    "Kurt Cobain a décrit son style musical comme une combinaison de punk rock, de pop et de heavy metal. Cette fusion unique a créé un son qui était à la fois accessible et brut.",
    "Nirvana a été l'un des premiers groupes à signer avec le label Geffen, ce qui a facilité leur ascension vers la célébrité. 'Nevermind' a atteint le sommet des charts, en grande partie grâce au succès de 'Smells Like Teen Spirit'.",
    "Avant de fonder Nirvana, Kurt Cobain faisait partie de plusieurs groupes locaux, mais c'est avec Nirvana qu'il a trouvé son véritable son et son identité musicale.",
    "Les premières performances de Nirvana étaient souvent brutales et bruyantes, mais c'est cette énergie sauvage qui a attiré l'attention de la scène musicale alternative.",
    "L'impact de Kurt Cobain et Nirvana sur la mode des années 90 est indéniable. Les chemises en flanelle, les jeans déchirés et les t-shirts de groupes sont devenus des éléments emblématiques du style grunge.",
    "Kurt Cobain a écrit la majeure partie des chansons de Nirvana, et sa capacité à capturer des émotions profondes et parfois conflictuelles dans sa musique a fait de lui un auteur-compositeur respecté.",
    "La chanson 'Heart-Shaped Box', extraite de l'album 'In Utero', est l'une des dernières œuvres majeures de Nirvana avant la mort de Kurt Cobain. Elle est souvent interprétée comme un reflet de ses luttes internes.",
    "L'influence de Kurt Cobain sur des groupes contemporains et ultérieurs est énorme. De nombreux artistes ont cité Nirvana comme une inspiration majeure, et leur musique continue de toucher les générations futures."
]

theme_ids = [f"cobain_doc_{i+1}" for i in range(len(theme_docs))]

theme_metas = [
    {"version": "1.0", "description": f"Document sur Kurt Cobain - Doc {i+1}", "created_by": "admin"}
    for i in range(len(theme_docs))
]

# Ajout des documents à la collection_KC2
collection_KC2.add(
    documents=theme_docs,
    ids=theme_ids,
    metadatas=theme_metas
)

# Fonction pour interroger la collection_KC2 sur un mot-clé ou une phrase
def search_collection(query):
    results = collection_KC2.query(query_texts=[query], n_results=3)
    print("\nRésultats de la recherche:")
    for result in results['documents']:
        print(f"- {result}")

# Fonction pour afficher les documents de la collection_KC2
def display_documents():
    print("\nDocuments dans la collection_KC2 :")
    for i, doc in enumerate(theme_docs, start=1):
        print(f"{i}. {doc}")

# Fonction principale de l'interface console
def main():
    while True:
        print("\n=== Menu ===")
        print("1. Ajouter de nouveaux documents")
        print("2. Afficher les documents")
        print("3. Rechercher dans les documents")
        print("4. Quitter")
        
        choice = input("Choisir une option (1/2/3/4) : ").strip()

        if choice == "1":
            # Ajouter un nouveau document
            new_doc = input("Entrez le texte du nouveau document sur Kurt Cobain : ").strip()
            new_id = f"cobain_doc_{len(theme_docs) + 1}"
            new_meta = {"version": "1.0", "description": "Nouveau document", "created_by": "admin"}
            
            theme_docs.append(new_doc)
            theme_ids.append(new_id)
            theme_metas.append(new_meta)
            
            # Ajouter à la collection_KC2
            collection_KC2.add(documents=[new_doc], ids=[new_id], metadatas=[new_meta])
            print("Document ajouté avec succès.")

        elif choice == "2":
            display_documents()

        elif choice == "3":
            # Rechercher dans les documents
            query = input("Entrez le terme de recherche : ").strip()
            search_collection(query)

        elif choice == "4":
            print("Au revoir !")
            break

        else:
            print("Option invalide, veuillez essayer de nouveau.")

if __name__ == "__main__":
    main()

Mini‑app de recherche (collection_KC2: kurt_cobain_2 )

=== Menu ===
1. Ajouter de nouveaux documents
2. Afficher les documents
3. Rechercher dans les documents
4. Quitter
Document ajouté avec succès.

=== Menu ===
1. Ajouter de nouveaux documents
2. Afficher les documents
3. Rechercher dans les documents
4. Quitter

Documents dans la collection_KC2 :
1. Kurt Cobain, le leader du groupe Nirvana, est souvent perçu comme une icône de la musique grunge. Né en 1967 à Aberdeen, Washington, il a fondé Nirvana avec le bassiste Krist Novoselic.
2. L'album 'Nevermind', sorti en 1991, a propulsé Nirvana au rang de superstars internationales. La chanson 'Smells Like Teen Spirit' est rapidement devenue un hymne générationnel.
3. Kurt Cobain a toujours revendiqué une approche musicale honnête et brute, où il exprimait ses émotions personnelles. La scène musicale grunge a explosé grâce à des groupes comme Nirvana, Pearl Jam, et Soundgarden.
4. Le titre 'Smells Like Teen Spirit' est l'un des morceaux

---
## Annexes

### Utilisation d'un autre embedder

Par défaut chromadb fait appel à l'embedder `sentence-transformers/all-MiniLM-L6-v2`.  
cf. https://docs.trychroma.com/docs/embeddings/embedding-functions#default-all-minilm-l6-v2

In [55]:
# Set your OPENAI_API_KEY environment variable
import chromadb.utils.embedding_functions as embedding_functions

huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="YOUR_API_KEY",
    model_name="sentence-transformers/all-distilroberta-v1"
)

from chromadb.utils.embedding_functions import HuggingFaceEmbeddingFunction

# Initialiser le client ChromaDB
client = chromadb.Client()

# Créer la fonction d'embedding avec un autre modèle Hugging Face
huggingface_ef = HuggingFaceEmbeddingFunction(
    model_name="paraphrase-MiniLM-L6-v2",  # Changer de modèle
    api_key="YOUR_HUGGINGFACE_API_KEY"  # Si nécessaire, ajoute ta clé API Hugging Face
)

# Créer une collection avec l'embedding Hugging Face
collection_test = client.create_collection(
    name="collection_embedding_test",
    embedding_function=huggingface_ef
)

# Exemple de documents
theme_docs = [
    "Kurt Cobain, le leader du groupe Nirvana, est souvent perçu comme une icône de la musique grunge. Né en 1967 à Aberdeen, Washington, il a fondé Nirvana avec le bassiste Krist Novoselic.",
    "L'album 'Nevermind', sorti en 1991, a propulsé Nirvana au rang de superstars internationales. La chanson 'Smells Like Teen Spirit' est rapidement devenue un hymne générationnel.",
    "Kurt Cobain a toujours revendiqué une approche musicale honnête et brute, où il exprimait ses émotions personnelles. La scène musicale grunge a explosé grâce à des groupes comme Nirvana, Pearl Jam, et Soundgarden.",
    "Le titre 'Smells Like Teen Spirit' est l'un des morceaux les plus emblématiques de Nirvana. Il est souvent cité comme l'une des chansons les plus importantes des années 1990.",
    "En plus de sa carrière musicale, Kurt Cobain était aussi un artiste visuel. Il a réalisé plusieurs dessins et peintures pendant sa vie.",
    "Kurt Cobain se battait contre des problèmes de santé mentale, y compris la dépression et l'anxiété, ce qui a influencé de nombreuses chansons de Nirvana.",
    "L'album 'In Utero', sorti en 1993, a marqué une évolution musicale pour Nirvana. Produit par Steve Albini, il est plus sombre et expérimental que 'Nevermind'.",
    "Le suicide de Kurt Cobain en avril 1994 a choqué le monde entier. Il a laissé derrière lui un héritage musical immense, mais aussi une série de questions non résolues sur sa santé mentale et ses luttes personnelles.",
    "Malgré sa disparition, l'impact de Kurt Cobain sur la musique et la culture populaire reste profond. Il est toujours considéré comme l'une des figures les plus importantes du rock des années 1990.",
    "Kurt Cobain était également un militant de la liberté d'expression et un critique acerbe de l'industrie musicale. Il a souvent critiqué l'exploitation commerciale de la musique.",
    "La chanson 'Come as You Are', de l'album 'Nevermind', est une des chansons les plus emblématiques de Nirvana. Elle évoque le concept de l'authenticité et de l'acceptation de soi.",
    "Kurt Cobain était marié à Courtney Love, la chanteuse de Hole. Leur relation a souvent été médiatisée et controversée, ce qui a ajouté une pression supplémentaire à sa vie personnelle.",
    "Le groupe Nirvana a joué un rôle clé dans la démocratisation du genre musical grunge, un mélange de punk et de rock alternatif, qui est devenu un phénomène mondial dans les années 90.",
    "Kurt Cobain a décrit son style musical comme une combinaison de punk rock, de pop et de heavy metal. Cette fusion unique a créé un son qui était à la fois accessible et brut.",
    "Nirvana a été l'un des premiers groupes à signer avec le label Geffen, ce qui a facilité leur ascension vers la célébrité. 'Nevermind' a atteint le sommet des charts, en grande partie grâce au succès de 'Smells Like Teen Spirit'.",
    "Avant de fonder Nirvana, Kurt Cobain faisait partie de plusieurs groupes locaux, mais c'est avec Nirvana qu'il a trouvé son véritable son et son identité musicale.",
    "Les premières performances de Nirvana étaient souvent brutales et bruyantes, mais c'est cette énergie sauvage qui a attiré l'attention de la scène musicale alternative.",
    "L'impact de Kurt Cobain et Nirvana sur la mode des années 90 est indéniable. Les chemises en flanelle, les jeans déchirés et les t-shirts de groupes sont devenus des éléments emblématiques du style grunge.",
    "Kurt Cobain a écrit la majeure partie des chansons de Nirvana, et sa capacité à capturer des émotions profondes et parfois conflictuelles dans sa musique a fait de lui un auteur-compositeur respecté.",
    "La chanson 'Heart-Shaped Box', extraite de l'album 'In Utero', est l'une des dernières œuvres majeures de Nirvana avant la mort de Kurt Cobain. Elle est souvent interprétée comme un reflet de ses luttes internes.",
    "L'influence de Kurt Cobain sur des groupes contemporains et ultérieurs est énorme. De nombreux artistes ont cité Nirvana comme une inspiration majeure, et leur musique continue de toucher les générations futures."
]

# Ajouter les documents à la collection
collection_test.add(
    documents=theme_docs,
    ids=[f"doc_{i}" for i in range(len(theme_docs))],
    metadatas=[{"source": "HuggingFace Embeddings"}] * len(theme_docs)
)

# Fonction pour interroger la collection
def search_collection(query_text):
    # Effectuer la recherche avec le texte de la requête
    results = collection.query(
        query_texts=[query_text],  # Le texte que l'on veut interroger
        n_results=3  # Nombre de résultats à obtenir
    )
    
    # Afficher les résultats
    print("\nRésultats de la recherche :")
    for i, doc in enumerate(results['documents'], start=1):
        print(f"{i}. {doc}")

# Test de la fonction de recherche
search_collection("Kurt Cobain musique")

ValueError: could not convert string to float: 'error' in add.

In [ ]:
# Définisez un nouvel embedder local (ex: nomic-embed-text:latest de ollama)
from chromadb.utils import embedding_functions
# TODO

In [ ]:
# Générez les embeddings
# TODO

# Créez une nouvelle collection avec les nouveaux embeddings 
# TODO

In [ ]:
# Alternative avec embedder
# Intégrez la fonction d'embedding lors de la création de la collection
# TODO

In [ ]:
# Testez une requête similaire à la précédente collection
# TODO

### Bonnes pratiques (à discuter / compléter)
- Utiliser `chromadb.PersistentClient()` pour conserver l'index entre sessions.
- Choisir un modèle d'embeddings adapté (local vs API) ; garder la cohérence entre index et requêtes.
- Pré‑traiter les textes (nettoyage, normalisation, segmentation en *chunks* si longs).
- Ajouter des métadonnées utiles dès l’ingestion (source, date, catégorie, langue).
- Tester et itérer : qualifier la pertinence via des jeux de requêtes réels.
- Surveiller le volume de données et la latence ; adapter le backend si besoin.


---
## Références utiles (à consulter)
- Documentation Chroma (Getting Started, API) : https://docs.trychroma.com/
- Répertoire GitHub : https://github.com/chroma-core/chroma
- Concepts embeddings (Sentence-Transformers) : https://www.sbert.net/
    
> **Note** : Vérifiez la version de `chromadb` et les éventuels changements d'API.
